In [1]:
import matlab.engine
import os
from tqdm import tqdm
import pandas as pd
import scipy.io as sio

In [2]:
calibrationDataFolder = r'C:\Users\romai\Documents\Zoom\calib'

In [13]:
eng = matlab.engine.start_matlab()

all_values = []

for calibFolder in tqdm(os.listdir(calibrationDataFolder)):
    
    for wavelength in os.listdir(os.path.join(calibrationDataFolder, calibFolder)):
        if wavelength.endswith('nm'):
            if len(os.listdir(os.path.join(calibrationDataFolder, calibFolder, wavelength))) == 0:
                pass
            else:
                if 'values.mat' in os.listdir(os.path.join(calibrationDataFolder, calibFolder, wavelength)):
                    pass
                else:
                    with open('tmp/current_folder_calib.txt', 'w') as f:
                        f.write(os.path.join(calibrationDataFolder, calibFolder, wavelength))
                    eng.calibration_polarimetre_Mueller(nargout=0)
                    
                mat_contents = sio.loadmat(os.path.join(calibrationDataFolder, calibFolder, wavelength, 'values.mat'))
                val = mat_contents['val'][0]
                values = {}
                values['meanKappaW'] = val[0]
                values['stdKappaW'] = val[1]
                values['meanKappaA'] = val[2]
                values['stdKappaA'] = val[3]
                values['meanKappaB'] = val[4]
                values['stdKappaB'] = val[5]
                values['folder'] = os.path.join(calibrationDataFolder, calibFolder)
                values['wavelength'] = wavelength

                all_values.append(values)
        else:
            pass
        
eng.quit()

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 482.96it/s]


In [30]:
wavelengthts = ['450nm', '500nm', '550nm', '600nm','650nm', '700nm']
df = pd.DataFrame.from_records(all_values)
df.set_index('folder', inplace=True)
df = df.iloc[::-1]

In [51]:
df_wls = {}
for wavelength in wavelengthts:
    df_wl = df[df['wavelength'] == wavelength].drop('wavelength', axis=1)
    df_wls[wavelength] = df_wl

In [52]:
with pd.ExcelWriter(os.path.join(os.getcwd().split('\\notebooks')[0], 'results', 'parameters.xlsx'), engine='xlsxwriter') as writer:
    for wavelength in wavelengthts:
        df_wls[wavelength].to_excel(writer, sheet_name=wavelength, index=False)